In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn import preprocessing

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf

In [ ]:
data = pd.read_csv('../input/weather-dataset-rattle-package/weatherAUS.csv')

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.isnull().sum()

In [ ]:
data.info

In [ ]:
data.describe()

In [ ]:
data.dtypes

In [ ]:
minimum_temperature = data['MinTemp'].min()
print(minimum_temperature)

In [ ]:
maximum_temperature = data['MaxTemp'].max()
print(maximum_temperature)

In [ ]:
data['Sunshine'].value_counts()

In [ ]:
data.hist(figsize=(20,15))

# **With the above histograms we can conclude:**

* WindSpeed at 9am and 3pm: The WindSpeed in morning high and during afternoon it's decreased
* Humidity at 9am and 3pm: Humidity is more in morning hours and afternoon it decrease
* Pressure at 9am and 3pm: The air pressure between the hours is same
* Cloud at 9am and 3pm: There's no significance difference between the clouds.
* Temp at 9am and 3pm: The temperature has increased in the duration of 6 hours

In [ ]:
print('Percentage of missing values: ')
print((data.isnull().sum().sort_values(ascending=False) / len(data)) * 100)

In [ ]:
missing_counts = data.isnull().sum().sort_values(ascending=True)
missing_counts.plot.barh(figsize=(10,8), title = 'Total number of missing values by attribute');

In [ ]:
data['RainToday'].mode()

In [ ]:
data['RainToday'].value_counts()

In [ ]:
data['RainToday'] = data['RainToday'].fillna('No')

In [ ]:
data['RainTomorrow'].value_counts()

In [ ]:
corr_matrix = data.corr()
corr_matrix

In [ ]:
temp = data[['MinTemp', 'MaxTemp', 'Temp9am', 'Temp3pm']]
temp.corr()

In [ ]:
from pandas.plotting import scatter_matrix
scatter_matrix(temp, figsize=(15,12), alpha=0.05, s=5);

In [ ]:
scatter_matrix(temp, figsize=(15,10), alpha=0.1);

In [ ]:
windspeed = data[['WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm']]
windspeed.corr()

In [ ]:
scatter_matrix(windspeed, figsize=(15,10), alpha=0.1);

In [ ]:
humid = data[['Humidity9am', 'Humidity3pm']]
humid.corr()

In [ ]:
scatter_matrix(humid, figsize=(12,8), alpha=0.02);

In [ ]:
import seaborn as sns

In [ ]:
sns.displot(data = data, kind = 'hist', x = 'Humidity9am', hue = 'RainTomorrow', multiple = 'stack',bins=25,height = 4, aspect = 1.7)


In [ ]:
sns.displot(data = data, kind = 'hist', x = 'Humidity3pm', hue = 'RainTomorrow', multiple = 'stack',bins=25,height = 4, aspect = 1.7)


In [ ]:
sns.displot(data = data, kind = 'hist', x = 'Cloud3pm', hue = 'RainTomorrow', multiple = 'stack',bins=25,height = 4, aspect = 1.7)


In [ ]:
sns.displot(data = data, kind = 'hist', x = 'Cloud9am', hue = 'RainTomorrow', multiple = 'stack',bins=25,height = 4, aspect = 1.7)


In [ ]:
sns.displot(data = data, kind = 'hist', x = 'Pressure3pm', hue = 'RainTomorrow', multiple = 'stack',bins=25,height = 4, aspect = 1.7)


In [ ]:
sns.displot(data = data, kind = 'hist', x = 'Pressure9am', hue = 'RainTomorrow', multiple = 'stack',bins=25,height = 4, aspect = 1.7)


In [ ]:
data = data.drop(columns=['Evaporation','Sunshine','Cloud3pm','Cloud9am','Date','Location'],
                 axis=1)

In [ ]:
data = data.dropna(how='any')
print(data.shape)

In [ ]:
# Replace No and Yes for 0 and 1 in RainToday and RainTomorrow
data['RainToday'].replace({'No': 0, 'Yes': 1},inplace = True)
data['RainTomorrow'].replace({'No': 0, 'Yes': 1},inplace = True)

# Categorical variables WindGustDir, WindDir3pm and WindDir9am in dummy variables for each category.
categoric_c = ['WindGustDir', 'WindDir3pm', 'WindDir9am']
datafinal = pd.get_dummies(data, columns=categoric_c)
print(datafinal.shape)
datafinal.head()

In [ ]:
standa = preprocessing.MinMaxScaler()
standa.fit(datafinal)
datafinal = pd.DataFrame(standa.transform(datafinal), index=datafinal.index, columns=datafinal.columns)
datafinal.head()

In [ ]:
# Calculate the correlation matrix
corr = datafinal.corr()
corr1 = pd.DataFrame(abs(corr['RainTomorrow']),columns = ['RainTomorrow'])
nonvals = corr1.loc[corr1['RainTomorrow'] < 0.005]
print('Var correlation < 0.5%',nonvals)
nonvals = list(nonvals.index.values)

In [ ]:

datafinal1 = datafinal.drop(columns=nonvals,axis=1)
print('Data Final',datafinal1.shape)

## Data for training and testing¶
Next, we proceed to select a set of training data that will be the input in the learning algorithms, to make sure that after training our classification algorithm it is able to generalize well to new data. For this case we use a test sample size of 20%.

In [ ]:
from sklearn.model_selection import train_test_split
Y = datafinal1['RainTomorrow']
X = datafinal1.drop(columns=['RainTomorrow'])
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=9)

In [ ]:
print('X train shape: ', X_train.shape)
print('Y train shape: ', Y_train.shape)
print('X test shape: ', X_test.shape)
print('Y test shape: ', Y_test.shape)

## Light GBM classification[](http://)

**Light GBM is a fast, distributed, high-performance gradient boosting framework based on decision tree algorithm, used for ranking, classification and many other machine learning tasks**

In [ ]:
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import accuracy_score
model = lgb.LGBMClassifier()
model.fit(X_train, Y_train)

In [ ]:
predict_train = model.predict(X_train)
print('\nTarget on train data',predict_train) 

In [ ]:
# Accuray Score on train dataset
accuracy_train = accuracy_score(Y_train,predict_train)
print('\naccuracy_score on train dataset : ', accuracy_train)
predict_test = model.predict(X_test)
print('\nTarget on test data',predict_test) 


In [ ]:

# Accuracy Score on test dataset']
accuracy_test = accuracy_score(Y_test,predict_test)
print('\naccuracy_score on test dataset : ', accuracy_test)

## Please Upvote if it helps